In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/shwetank/code/wiki-base')
import torch
# from datasets import load_dataset
# import tiktoken

# Setup device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

In [ ]:
# Hyperparams
batch_size = 32
block_size = 128
emb_dim = 384
num_heads = 6
num_layers = 4
dropout = 0.2
linear_scaleup = 4
temperature = 1

# Optimization loop parameters
n_epochs = 100
eval_interval =10

In [ ]:
with open('./data/shakespeare/input.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

n1 = int(0.9*len(text_data))
train_data = text_data[:n1]
val_data = text_data[n1:]
vocab = sorted(set(list(text_data)))
n_vocab = len(vocab)
print(f"vocab: {vocab}")
print(f"vocab length: {n_vocab}")

In [ ]:
from torch.utils.data import Dataset
import torch

class CharacterDataset(Dataset):
    def __init__(self, dataset: Dataset, block_size: int=128, batch_size: int=16):
        self.block_size = block_size
        self.dataset = dataset
        self.batch_size = batch_size
        

    def __len__(self) -> int:
        return self.block_size

    def __getitem__(self, idx: int) -> tuple[torch.tensor, torch.tensor]:
        x = torch.zeros((self.block_size + 1), dtype=torch.long)
        y = torch.zeros((self.block_size + 1), dtype=torch.long)
        # print(idx)
        x = self.dataset[idx:idx+self.block_size]
        y = self.dataset[idx+1:idx+self.block_size+1]
        return x.to(device), y.to(device)

In [ ]:
stoi = {s: i for i,s in enumerate(vocab)}
itos = {i:s for s,i in stoi.items()}
# print(stoi)
# print(itos)
def encode(text):
    tokens = torch.tensor([stoi[t] for t in text])
    return tokens

def decode(tokens):
    text = ''.join(itos[t] for t in tokens)
    return text

# tokens = encode('Hello world!')
# print(decode(tokens))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Feedforward(nn.Module):
    def __init__(self,emb_dim,dropout) -> None:
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, linear_scaleup * emb_dim),
            nn.ReLU(),
            nn.Linear(linear_scaleup * emb_dim, emb_dim),
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.ff(x)

class Block(nn.Module):
    def __init__(self, emb_dim, num_heads, dropout) -> None:
        super().__init__()
        self.head_size = emb_dim // num_heads
        self.sa_head = nn.MultiheadAttention(emb_dim, num_heads, dropout)
        self.ff = Feedforward(emb_dim, dropout)
        self.ln1 = nn.LayerNorm(emb_dim)
        self.ln2 = nn.LayerNorm(emb_dim)

    def forward(self, x, targets=None):
        x = self.ln1(x)
        sa_out, _ = self.sa_head(x,x,x)
        x = x + sa_out
        x = x + self.ff(self.ln2(x))
        return x


class Xformer(nn.Module):
    def __init__(self, emb_dim, n_vocab, num_heads, num_layers, dropout): 
        super().__init__()
        self.tok_emb = nn.Embedding(n_vocab, emb_dim)
        self.pos_emb = nn.Embedding(n_vocab, emb_dim)
       
        blocks = []
        for _ in range(num_layers):
            blocks.append(Block(emb_dim, num_heads, dropout))
        
        self.blocks = nn.Sequential(*blocks, nn.LayerNorm(emb_dim))
        self.lm_head = nn.Linear(emb_dim, n_vocab)

    def forward(self, x, targets=None):
        x = self.tok_emb(x) + self.pos_emb(x)
        x = self.blocks(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            # print(logits.view(-1, logits.size(-1)).shape, targets.view(-1).shape)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

        return(logits,loss)

In [ ]:
## Get dataloader for train and  val datasets from training and test data
# from torch.utils.data import DataLoader
# train_data_obj = WikiData(train_tensor, block_size=block_size)
# val_data_obj = WikiData(val_tensor, block_size=block_size)
# train_dataloader = DataLoader(train_data_obj, batch_size=1)
# val_dataloader = DataLoader(val_data_obj, batch_size=1)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(CharacterDataset(encode(train_data)),batch_size=batch_size)
val_dataloader = DataLoader(CharacterDataset(encode(val_data)), batch_size=batch_size)

In [ ]:
# Single test run
from torch.optim import Adam
xb, yb = next(iter(train_dataloader))
# print(xb.shape, yb.shape)
model = Xformer(emb_dim, n_vocab, num_heads, num_layers, dropout).to(device)
optimizer = Adam(model.parameters(), lr=0.001)
logits, loss = model(xb,yb)
print(loss)

In [ ]:
from utils import get_model_size
get_model_size(model)

In [ ]:
# ## Optimal lr sweep
# from utils import get_lr_loss
# import matplotlib.pyplot as plt
# num_epochs = 100
# lri, lossi =  get_lr_loss(model, optimizer, train_dataloader, num_epochs, device, -5, -3)
# plt.plot(lri, lossi)
# # Add labels to the x-axis and y-axis
# plt.xlabel('LR (Learning Rate)')
# plt.ylabel('Loss')

In [ ]:
tr_loss = []
vl_loss = []

In [ ]:
from utils import evaluate_loss
lr = 1e-4
optimizer = Adam(model.parameters(), lr=lr)
for steps in range(100):
    xb, yb = next(iter(train_dataloader))
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    tr_lossi, te_lossi = evaluate_loss(model, train_dataloader, val_dataloader, device, num_batches=10)
    tr_loss.append(tr_lossi)
    vl_loss.append(te_lossi)
    # print(loss.item())

    if steps % eval_interval-1 == 0:
        print('ltr: ', tr_lossi, 'lte: ', te_lossi, 'single shot loss:', loss.item())

In [ ]:
## Plot loss 
import matplotlib.pyplot as plt

plt.figure()
plt.plot(tr_loss, label='Training Loss')
plt.figure()
plt.plot(vl_loss, label='Validation Loss')
plt.legend()
plt.show()
print('training loss: ', round(torch.mean(torch.tensor(tr_loss[-10:])).item(),4)), 
print('validation loss: ', round((torch.mean(torch.tensor(vl_loss[-10:]))).item(),4))

In [ ]:
@torch.no_grad()
def generate(model, max_new_tokens=block_size, batch_size=batch_size, temperature=temperature):
    idx = torch.zeros((batch_size, 1), dtype=torch.long).to(device)
    for _ in range(max_new_tokens):
        # print('idx shape:',idx.shape)
        idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
        idx_cond = idx_cond.to(device)
        logits, _ = model(idx_cond)
        # Pick only the logits from most recent time step. Karpathy also does a divide by temp?
        # This is just Platt scaling which makes the various Softmax curves closes adding more randomness
        # see scratch.ipynb. https://en.wikipedia.org/wiki/Platt_scaling
        logits = logits[:,-1,:]/temperature
        probs = F.softmax(logits, dim=-1)
        # print('prob dist:',probs)
        idx_next = torch.multinomial(probs, num_samples=1)
        # print('idx_next shape:',idx_next.shape)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx


In [ ]:
X_samp = generate(model, block_size, 1, temperature).tolist()

In [ ]:
for row in X_samp:
    print(tokenizer.decode(row))